In [ ]:
#ipcluster start -n=8

In [1]:
%pylab inline
import time
import numexpr as ne
import seaborn as sns
from ipywidgets import interact
from sklearn.decomposition import PCA

from numba import jit

Populating the interactive namespace from numpy and matplotlib


In [2]:
ne.set_num_threads(32)

8

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%gui qt

In [5]:
import sys
sys.path.append('../../..')

In [6]:
from spiketag.mvc import Sorter
from spiketag.base import ProbeFactory, MUA, bload, SPKTAG
from spiketag.utils import conf

# Test 32 liner probe

In [ ]:
nCh = 32
fs = 25000
probe = ProbeFactory.genLinearProbe(fs=fs, n_ch=nCh)

In [ ]:
# app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin',probe = probe, 
#               corr_cutoff=0.95, 
#               fetlen=6, 
#               fet_whiten=False,
#               fall_off_size=18, 
#               n_jobs=1)
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin', 
             spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.probe')



In [ ]:
#app.model.spktag.tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_32_25k.bin.probe')

In [ ]:
app.run()

# Switch of profiler log, if True, print profiler log

In [ ]:
from spiketag.utils import conf

In [ ]:
conf.ENABLE_PROFILER = True

# Use line profiler to find bottomneck

In [ ]:
app.model.clu[30].selectlist

In [ ]:
app.model.clu[30].index

In [ ]:
# pip install line-profiler first.
%load_ext line_profiler

In [ ]:
%lprun -f app.view.trace_view.locate_and_highlight app.view.trace_view.locate_and_highlight(app.view.spk_view.selected_spk)

# Test 160 tetrode probe

In [7]:
nCh = 64
fs = 25e3
tetrode = ProbeFactory.genTetrodeProbe(fs=fs, n_ch=nCh)

In [8]:
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_64_25K_07_10_2017.bin',probe = tetrode, 
              corr_cutoff=0.95, 
              fet_method='pca',
              fetlen=6, 
              fet_whiten=False,
              fall_off_size=18, 
              n_jobs=1)
#app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin', spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_160_30K_bank0and1_25:5:2017.bin.probe')

2017-07-10 12:35:16,094 - spiketag - INFO - load mua data
2017-07-10 12:35:16,109 - spiketag - INFO - #############  load data  ###################
2017-07-10 12:35:16,111 - spiketag - INFO - /Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_64_25K_07_10_2017.bin loaded, it contains: 
2017-07-10 12:35:16,113 - spiketag - INFO - 7361618 * 64 points (1884574208 bytes) 
2017-07-10 12:35:16,114 - spiketag - INFO - 64 channels with sampling rate of 25000.0000 
2017-07-10 12:35:16,116 - spiketag - INFO - 294.465 secs (4.908 mins) of data
2017-07-10 12:35:16,118 - spiketag - INFO - #############################################
2017-07-10 12:35:17,281 - spiketag - INFO - removing high corr noise from spikes pool
2017-07-10 12:35:23,683 - spiketag - INFO - removed noise ids: [] 
2017-07-10 12:35:23,685 - spiketag - INFO - removing all spks on group which len(spks) less then fetlen
2017-07-10 12:35:23,697 - spiketag - INFO - removed all spks on these groups: {0: 1, 9: 2, 2: 4}
2

In [ ]:
app.model.spktag.tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_64_25K_07_10_2017.bin.probe')

In [9]:
app.run()

2017-07-10 12:35:41,882 - spiketag - WARNING -  channel 0 has no spikes! 
2017-07-10 12:40:05,415 - spiketag - INFO - received model modified event, removed spike[group=14, global_id=[912]]
2017-07-10 12:40:22,471 - spiketag - INFO - received model modified event, removed spike[group=14, global_id=[912]]


### Detect peak and output pivotal file

In [ ]:
bf = bload(nCh=64, fs=25000)

In [ ]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/tt16bit_160.bin', dtype='int16')

In [ ]:
spks = bf.detect_spks()

In [ ]:
data = bf.asarray()

In [ ]:
from ipywidgets import interact

In [ ]:
@interact(x=(1000,55752704,100), y=(0,63,1))
def plot_wave(x,y):
    plot(data[x-1000:x,y])

In [ ]:
from spiketag.view import wave_view

In [ ]:
data = bf.npmm.reshape(-1,64).astype(np.float32)

In [ ]:
wv = wave_view(data=data, spks=spks)

In [ ]:
wv.show()